<a href="https://colab.research.google.com/github/southstone0201/Transformer/blob/main/dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("iwslt2017",'iwslt2017-en-de')

Generating train split:   0%|          | 0/206112 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8079 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/888 [00:00<?, ? examples/s]

In [ ]:
import sentencepiece as spm

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 206112
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8079
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 888
    })
})


In [ ]:
with open("iwslt2017-en.txt", "w") as f_en, open("iwslt2017-de.txt", "w") as f_de:
    for it in dataset["train"]:
        f_en.write(it["translation"]["en"] + "\n")
        f_de.write(it["translation"]["de"] + "\n")

In [ ]:
with open('iwslt2017-en.txt', 'r') as f:
    en = f.readlines()
with open('iwslt2017-de.txt', 'r') as f:
    de = f.readlines()

In [ ]:
import pandas as pd
train_df=pd.DataFrame()
train_df['en']=pd.DataFrame(en)
train_df['de']=pd.DataFrame(de)

In [ ]:
train_df

,en,de
0,"Thank you so much, Chris.\n","Vielen Dank, Chris.\n"
1,And it's truly a great honor to have the oppor...,"Es ist mir wirklich eine Ehre, zweimal auf die..."
2,"I have been blown away by this conference, and...",Ich bin wirklich begeistert von dieser Konfere...
3,"And I say that sincerely, partly because I ne...","Das meine ich ernst, teilweise deshalb -- weil..."
4,Put yourselves in my position.\n,Versetzen Sie sich mal in meine Lage!\n
...,...,...
206107,The Ancient Greeks didn't just wake up one day...,Den alten Griechen fiel auch nicht eines Tages...
206108,"It took centuries, even, for humans to realize...","Wir Menschen brauchten Jahrhunderte, um Sachen..."
206109,And so we must continuously challenge our noti...,Deshalb müssen wir unser Verständnis von Norma...
206110,Characters: Thank you. Thank you. Thank you. T...,Figuren: Danke. Danke. Danke. Danke. Danke.\n


In [ ]:
spm.SentencePieceTrainer.Train(
    input='iwslt2017-en.txt',
    model_prefix='en-sp',
    vocab_size=10000,
    model_type='bpe' ,
    )

In [ ]:
sp_en = spm.SentencePieceProcessor(model_file='./en-sp.model')

In [ ]:
train_df['en_encoded']=train_df['en'].apply(lambda x:sp_en.encode(x))



In [ ]:
spm.SentencePieceTrainer.train(
    input='iwslt2017-de.txt',
    model_prefix='de-sp',
    model_type="bpe",
    vocab_size=10000,
    )

In [ ]:
sp_de = spm.SentencePieceProcessor(model_file='./de-sp.model')

In [ ]:
train_df['de_encoded']=train_df['de'].apply(lambda x:sp_de.encode(x))

In [ ]:
print(train_df)

                                                       en  ...                                         de_encoded
0                             Thank you so much, Chris.\n  ...                      [1269, 662, 9937, 3281, 9940]
1       And it's truly a great honor to have the oppor...  ...  [183, 61, 336, 402, 113, 8264, 9937, 6815, 116...
2       I have been blown away by this conference, and...  ...  [145, 502, 402, 5014, 109, 337, 4789, 9937, 35...
3       And I say that sincerely, partly because  I ne...  ...  [179, 408, 79, 3000, 9937, 7066, 2511, 287, 41...
4                        Put yourselves in my position.\n  ...     [218, 1310, 95, 138, 940, 56, 408, 1967, 9995]
...                                                   ...  ...                                                ...
206107  The Ancient Greeks didn't just wake up one day...  ...  [1273, 2742, 8731, 5195, 250, 104, 505, 2086, ...
206108  It took centuries, even, for humans to realize...  ...  [186, 278, 5417, 1624, 2

In [ ]:
max_en_len=max(len(encoded) for encoded in train_df['en_encoded'])
max_de_len=max(len(encoded) for encoded in train_df['de_encoded'])
max_len=max(max_en_len,max_de_len)

In [ ]:
for en_sentence, de_sentence in zip(train_df['en_encoded'],train_df['de_encoded']):
    while len(en_sentence) < max_len:
        en_sentence.append(0)
    while len(de_sentence) < max_len:
        de_sentence.append(0)

In [ ]:
from torch.utils.data import Dataset,DataLoader
import torch


In [ ]:
class CustomDataset(Dataset):
  def __init__(self,en,de):
    self.en=torch.tensor(en)
    self.de=torch.tensor(de)
  def __len__(self):
    return len(self.en)
  def __getitem__(self, index):
      return self.en[index], self.de[index]

In [ ]:
train_dataset=CustomDataset(train_df['en_encoded'],train_df['de_encoded'])

In [ ]:
train_dataloader=DataLoader(train_dataset,batch_size=16,shuffle=True)